In [835]:
#pip install img2vec_pytorch

In [1]:
#importing libraries
import os
import numpy 
from img2vec_pytorch import Img2Vec
from PIL import Image
import IPython.display as display
import json
import glob
import shutil
import pandas as pd

In [ ]:
from img2vec_pytorch import Img2Vec
from PIL import Image

# Initialize Img2Vec with GPU
img2vec = Img2Vec(cuda=True,model = 'resnet152', layer_output_size=2048)

In [69]:
#Extracting information from image and json files
exceptions = []
file_name=[]
df =[]
data_df=[]
for idx,f in enumerate(os.listdir("/home/amisha/json_10/")):
    image_path = "/home/amisha/json_10/"+f
    with open(image_path) as j:
        data = pd.read_json(j)
        data['image_id'] = idx+1
        data['image_path'] = image_path
        data = pd.DataFrame(data)
        data = data.dropna()
        data_df.append(data)
        try:
            file=dict({'filename':f})
            df.append(file)  
        except:
            exceptions.append(f)

In [ ]:
data_df = pd.concat(data_df)

In [72]:
data_df.columns = ['text', 'bbox', 'label','image_id', 'image_path', 'new_col4','new_col5']

In [120]:
data_df["label"].unique()

array(['abstract', 'journal', 'title', 'author', 'affiliation', 'date',
       'doi'], dtype=object)

In [114]:
data_df=data_df[(data_df.label == "abstract") | (data_df.label == "title")|(data_df.label == "date")|(data_df.label == "author")|(data_df.label == "affiliation")|(data_df.label == "journal")|(data_df.label == "doi")|(data_df.label == "Address")]

In [116]:
data_df1 = data_df.drop(['new_col4','new_col5'], axis = 1)

In [117]:
data_df1 = data_df1.reset_index(drop=True)
data_df1.index += 1 

In [118]:
data_df1["label"].value_counts()

title          31303
author         19747
affiliation    15590
abstract        6101
journal         4426
date            3525
doi              440
Name: label, dtype: int64

In [16]:
data_df1.head()

,text,bbox,label,image_id,image_path
1,Abstract. The article is about the specifics o...,"[142.0, 951.0, 1479.0, 1097.0]",abstract,1,/home/amisha/json_10/Обусловленность политичес...
2,2017|05,"[578.0, 956.0, 745.0, 1007.0]",journal,2,/home/amisha/json_10/Eingabenkultur und Diktat...
3,Eingabenkultur und Diktaturerfahrungen in der DDR,"[582.0, 1156.0, 1438.0, 1296.0]",title,2,/home/amisha/json_10/Eingabenkultur und Diktat...
4,"Cornelia Neuert, Patricia Hadler, Timo Lenzner...","[576.0, 1653.0, 1605.0, 1818.0]",author,2,/home/amisha/json_10/Eingabenkultur und Diktat...
5,Palliative care and bioethics: study with assi...,"[223.0, 657.0, 1143.0, 695.0]",title,3,/home/amisha/json_10/Palliative care and bioet...


In [17]:
#hot encoding for the label
openI1 = pd.get_dummies(data_df1["label"])

In [18]:
from sklearn.model_selection import train_test_split

training_data, testing_data = train_test_split(openI1, test_size=0.2, random_state=25,shuffle=True,stratify = data_df1["label"].values)

print(f"No. of training examples: {training_data.shape[0]}")
print(f"No. of testing examples: {testing_data.shape[0]}")

No. of training examples: 64553
No. of testing examples: 16139


In [19]:
training_data.loc[:,'split'] = 'train'
testing_data.loc[:,'split'] = 'test'

/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1597: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = value
/usr/local/lib/python3.8/dist-packages/pandas/core/indexing.py:1720: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


In [20]:
openI1 = pd.concat([training_data, testing_data], axis=0)

In [23]:
openI1.reset_index(inplace=True)

In [24]:
openI1.rename(columns={'index':'id'}, inplace=True)

In [27]:
openI1.set_index("id",inplace=True)

In [28]:
a = openI1.sort_values("id")

In [29]:
openI = pd.concat([a,data_df1['text']],axis=1)

In [30]:
openI.reset_index(inplace=True)
openI = openI.rename(columns = {'index':'id'})

In [31]:
openI

,id,abstract,affiliation,author,date,journal,title,split,text
0,1,1,0,0,0,0,0,train,Abstract. The article is about the specifics o...
1,2,0,0,0,0,1,0,test,2017|05
2,3,0,0,0,0,0,1,train,Eingabenkultur und Diktaturerfahrungen in der DDR
3,4,0,0,1,0,0,0,test,"Cornelia Neuert, Patricia Hadler, Timo Lenzner..."
4,5,0,0,0,0,0,1,train,Palliative care and bioethics: study with assi...
...,...,...,...,...,...,...,...,...,...
80687,80688,0,1,0,0,0,0,train,la implementacio6n del Programa de Apoyo pleme...
80688,80689,1,0,0,0,0,0,train,*“* Profesor investigador en la Universidad Au...
80689,80690,0,0,0,0,0,1,train,"Griindung der ""Association of the Sociologists..."
80690,80691,0,0,1,0,0,0,train,Heine von Alemann


In [32]:
a=openI.loc[(openI["abstract"]==1) & (openI["split"]=="train")]
len(a)

4881

In [33]:
s=openI.loc[(openI["affiliation"]==1) & (openI["split"]=="train")]
len(s)

12472

In [34]:
s=openI.loc[(openI["title"]==1) & (openI["split"]=="train")]
len(s)

25042

In [35]:
a=openI.loc[(openI["author"]==1) & (openI["split"]=="train")]
len(a)

15797

In [36]:
a=openI.loc[(openI["date"]==1) & (openI["split"]=="train")]
len(a)

2820

In [37]:
a=openI.loc[(openI["journal"]==1) & (openI["split"]=="train")]
len(a)

3541

In [38]:
import pickle
# save dataframe to pickle file
with open('/home/amisha/Transformers-VQA/data/openI.pkl', 'wb') as file:
    pickle.dump(openI, file, protocol=pickle.HIGHEST_PROTOCOL)

In [39]:
#creating embedding vector with img2Vec
Name = []
Embeddings =[]
size=[]
BBox = []

for idx, row in data_df1.iterrows():
    BBox_row = row[1]
    bbox_np_arr = numpy.array([BBox_row],numpy.int32)
    BBox.append(bbox_np_arr)
    #print(BBox_row)

    name, extension = os.path.splitext(row[4])
    #print(name)
    
    img1 = Image.open("/home/amisha/images_10/"+name.split("/")[-1]+'.jpg')
    im_size = img1.size
    #print(im_size)
    
    size.append(im_size)
    cropped_im = img1.crop(row[1])

    vec = img2vec.get_vec(cropped_im, tensor=True)
    vec = vec.reshape(1,2048)
    np_arr = vec.cpu().detach().numpy()
    Embeddings.append(np_arr)

In [40]:
new_embed = []
for i in range(0,len(Embeddings)):
    new_embed.append(Embeddings[i][0])

In [41]:
data_df1['size'] = size

In [42]:
data_df1.index.name = 'id'

In [44]:
data_df1['Embeddings'] = Embeddings

In [45]:
data_df1['BBox'] = BBox

In [46]:
data_df2 = data_df1.drop(['text','image_path','bbox'],axis =1)

In [47]:
data_df2.reset_index(drop=True, inplace=True)

In [49]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()
data_df2["LABEL"] = labelencoder.fit_transform(data_df2["label"]) 

In [58]:
data_df2

,label,image_id,size,Embeddings,BBox,LABEL
0,abstract,1,"(1654, 2339)","[[0.1515294, 0.16800034, 0.09502802, 1.004601,...","[[142, 951, 1479, 1097]]",0
1,journal,2,"(1654, 2339)","[[0.17616051, 0.2546115, 0.00018553973, 0.0, 0...","[[578, 956, 745, 1007]]",4
2,title,2,"(1654, 2339)","[[0.02300931, 0.34190863, 0.26479852, 0.664131...","[[582, 1156, 1438, 1296]]",5
3,author,2,"(1654, 2339)","[[0.03216585, 0.34966055, 0.16534907, 1.040945...","[[576, 1653, 1605, 1818]]",2
4,title,3,"(1654, 2339)","[[2.7137752, 0.6436359, 0.09835813, 0.5155353,...","[[223, 657, 1143, 695]]",5
...,...,...,...,...,...,...
80687,affiliation,34999,"(1534, 2007)","[[1.6049753, 1.5060737, 0.13247229, 0.34662136...","[[232, 940, 1304, 966]]",1
80688,abstract,34999,"(1534, 2007)","[[0.18397199, 0.17473847, 0.50997806, 0.219659...","[[232, 1362, 1302, 1559]]",0
80689,title,35000,"(1046, 1487)","[[0.0027051086, 0.24816172, 0.05799809, 0.0517...","[[206, 172, 762, 301]]",5
80690,author,35000,"(1046, 1487)","[[1.996199, 1.1742235, 0.11424567, 0.34917492,...","[[382, 358, 585, 377]]",2


In [51]:
from sklearn.model_selection import train_test_split

# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(new_embed, y, test_size=0.2,random_state=109) # 70% training and 30% test

In [52]:
#Import svm model
from sklearn import svm

#Create a svm Classifier
clf = svm.SVC(kernel='linear') # Linear Kernel

#Train the model using the training sets
clf.fit(X_train, y_train)

#Predict the response for test dataset
y_pred = clf.predict(X_test)

In [54]:
len(y_pred)

24208

In [56]:
len(y_test)

24208

In [59]:
from sklearn.metrics import classification_report
target_names = ['abstract','affiliation', 'author','date','journal','title']
report = pd.DataFrame(classification_report(y_test, y_pred, target_names=target_names)).transpose()
print(classification_report(y_test, y_pred, target_names=target_names))

              precision    recall  f1-score   support

    abstract       0.68      0.58      0.62      1756
 affiliation       0.75      0.84      0.80      4683
      author       0.87      0.87      0.87      5865
        date       0.91      0.90      0.91      1039
     journal       0.86      0.78      0.82      1360
       title       0.88      0.87      0.88      9505

    accuracy                           0.84     24208
   macro avg       0.83      0.81      0.81     24208
weighted avg       0.84      0.84      0.84     24208



In [ ]:
report.to_csv('Result_SVM.csv', index= True)

In [52]:
#dictionary for each image correspondance to sub image
openI_v_f = {}

for i, row in data_df2.iterrows():
    #print(i)
    #openI_v_f[i] = [new[i]["BBox"], new[i]["Embeddings"], tuple(new[i]["size"].unique())]
    openI_v_f[i+1] = [(row["BBox"]), (row["Embeddings"]), row["size"]]

In [53]:
feature_example, bbox_example, (img_w_example, img_h_example)  = openI_v_f[openI.id.iloc[0]]

In [54]:
feature_example.shape, bbox_example.shape, (img_w_example, img_h_example)

((1, 4), (1, 2048), (1654, 2339))

In [55]:
import pickle
# save embedding vectors to pickle file
with open("/home/amisha/Transformers-VQA/data/openI_v_f.pkl", 'wb') as file:
    pickle.dump(openI_v_f, file, protocol=pickle.HIGHEST_PROTOCOL)